# Sprint 6 · Webinar 19 · Data Analytics teórico (Preparación de datos + distribuciones + Git/GitHub)


## Fecha

14 de diciembre de 2025


## Objetivos de la sesión teórica

Al finalizar la sesión, el/la estudiante podrá:

- Preparar un dataset “realista” para análisis estadístico: identificar variables relevantes, manejar valores ausentes/ inválidos y automatizar limpieza con Python.
- Usar estructuras básicas del lenguaje (iterables, bucles `for` / `while`) para recorrer y transformar datos.
- Escribir mini-funciones reutilizables de limpieza de datos (primera aproximación a “data pipelines” simples).
- Describir distribuciones numéricas y categóricas mediante estadísticas y visualizaciones (histogramas y boxplots).
- Traducir el comportamiento de una distribución a significado de negocio (qué implica para churn, tickets, gasto, etc.).
- Dar los primeros pasos en control de versiones: instalar Git, crear cuenta en GitHub y entender el flujo básico.

> Nota: este notebook está diseñado para principiantes. El código está comentado y se apoya en un dataset con “problemas” intencionales para practicar limpieza.


## Agenda sugerida (100 minutos)

1. Contexto del caso + dataset (5 min)  
2. Ejercicio 0: discusión guiada (breakout rooms) (10 min)  
3. Ejercicio 1: preparación de datos para análisis estadístico (30 min)  
4. Ejercicio 2: automatización con funciones + bucles (25 min)  
5. Ejercicio 3: describiendo distribuciones (estadística + visualización) (25 min)  
6. Cierre: Git/GitHub (instalación y primeros pasos) (5 min)  


## Ejercicio 0 · Calentamiento en breakout rooms (discusión conceptual, 10 min)

En grupos pequeños, discutan y respondan:

1. ¿Qué hace que un dataset sea “apto” para análisis estadístico? (pista: tipos de datos, valores faltantes, valores inválidos, sesgos)
2. Si tu objetivo fuera explicar **por qué algunos clientes hacen churn**, ¿qué variables buscarías primero y por qué?
3. Si ves valores como `age = 999` o `total_spend_90d_usd = -120`, ¿qué harías antes de “arreglarlo”?
4. ¿En qué casos **NO** conviene imputar valores faltantes con promedios?

Luego, compartan 2 conclusiones con el grupo.


## Ejercicio 1 · Preparación de datos para análisis estadístico (7.1)

En este ejercicio cargaremos un dataset “extenso” de clientes en LatAm (5.000 filas) con variables de comportamiento (compras, tickets, satisfacción), producto (plan), y resultado (churn).  
El dataset incluye valores faltantes e inválidos **intencionalmente** para practicar.

### 7.1.1 Identificando variables relevantes para el análisis

- Una **variable relevante** es aquella que ayuda a explicar o predecir un fenómeno de interés (p. ej., churn).  
- Se seleccionan variables por:
  - **Preguntas de negocio**: “¿Qué se asocia a churn?”  
  - **Disponibilidad y calidad**: si una columna es casi todo vacío o inconsistente, quizá no sirve (aún).
  - **Tipo de dato**: numérica vs categórica vs temporal.
  - **Riesgo de fuga de información (leakage)**: variables que revelan el resultado de forma directa (evitar).

Vamos a empezar con una exploración básica para ubicar columnas, tipos y problemas.


In [ ]:
# 1) Librerías base para el webinar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

# (Opcional) para ver más columnas en pantalla
pd.set_option('display.max_columns', 50)

# 2) Cargar el dataset (ajusta la ruta si es necesario)
DATA_PATH = "webinar19_customers_latam.csv"
df = pd.read_csv(DATA_PATH)

print("Filas y columnas:", df.shape)
df.head()


In [ ]:
# 3) Vista rápida de tipos de datos y valores faltantes
df.info()

# 4) Contar valores faltantes por columna (NaN)
missing_counts = df.isna().sum().sort_values(ascending=False)
missing_counts


### 7.1.2 Manejando valores ausentes o inválidos

En la práctica, no solo hay “faltantes”; también hay **inválidos** (ej.: edades negativas, fechas imposibles, montos negativos).

Estrategias típicas:

- **Eliminar** filas (si son pocas y no sesgan el análisis).
- **Reemplazar / imputar**:
  - numéricas: media/mediana (dependiendo de sesgo), o imputación por grupo (p. ej., por plan/segmento).
  - categóricas: “Unknown/No disponible”.
- **Corregir**: cuando el valor es inválido pero recuperable (p. ej., normalizar un texto).
- **Marcar**: crear columnas “flag” (por ejemplo `is_age_invalid`) para no perder información.

Primero detectemos valores claramente inválidos.


In [ ]:
# Reglas simples de validación (rules of thumb) para este caso
# Nota: Las reglas dependen del contexto. Aquí usaremos ejemplos razonables.

# 1) Edades inválidas: <= 0 o > 100
age_invalid_mask = (df["age"].isna()) | (df["age"] <= 0) | (df["age"] > 100)
print("Edad faltante o inválida:", age_invalid_mask.mean().round(3), "proporción")

In [ ]:
# 2) Gasto negativo en 90 días
spend_negative_mask = df["total_spend_90d_usd"] < 0
print("Gasto negativo:", spend_negative_mask.mean().round(3), "proporción")

In [ ]:
# 3) País / ciudad vacíos
country_empty_mask = df["country"].fillna("").str.strip().eq("")
city_empty_mask = df["city"].fillna("").str.strip().eq("")
print("Country vacío:", country_empty_mask.mean().round(3))
print("City vacío:", city_empty_mask.mean().round(3))

In [ ]:
# 4) Vista de algunos ejemplos problemáticos
df.loc[age_invalid_mask | spend_negative_mask | country_empty_mask | city_empty_mask].head(10)

### 7.1.3 Funciones de Python para automatizar la limpieza de datos

Antes de escribir “funciones propias”, aprovechamos funciones estándar:

- `pd.to_datetime(...)` para convertir fechas (con `errors="coerce"` para convertir inválidos a `NaT`).
- `pd.to_numeric(...)` para convertir numéricos (con `errors="coerce"` para convertir inválidos a `NaN`).
- `str.strip()`, `str.lower()` para normalizar texto.
- `fillna(...)`, `dropna(...)` para manejar faltantes.

Apliquemos esto a columnas “raw” que vienen como texto: `monthly_fee_raw` y `last_activity_raw`.


In [ ]:
# 1) Convertir monthly_fee_raw a numérico (los 'N/A' y 'unknown' se vuelven NaN)
df["monthly_fee_clean"] = pd.to_numeric(df["monthly_fee_raw"], errors="coerce")

In [ ]:
# 2) Convertir last_activity_raw a fecha (fechas imposibles -> NaT)
df["last_activity_clean"] = pd.to_datetime(df["last_activity_raw"], errors="coerce")

In [ ]:
# 3) Chequeos rápidos
print("monthly_fee_raw: NaN después de convertir:", df["monthly_fee_clean"].isna().mean().round(3))
print("last_activity_raw: NaT después de convertir:", df["last_activity_clean"].isna().mean().round(3))

df[["monthly_fee_raw", "monthly_fee_clean", "last_activity_raw", "last_activity_clean"]].head(10)

## Ejercicio 2 · Automatización con iterables, bucles y mini-funciones (7.1.4–7.1.5)

### 7.1.4 Bucles (`for` & `while`) e iterables en una colección

- Un **iterable** es una colección que puedes recorrer elemento por elemento (lista, diccionario, Series, etc.).
- `for` se usa cuando sabes que vas a recorrer una colección completa (o un rango).
- `while` se usa cuando la repetición depende de una condición (ej.: “mientras haya valores inválidos…”).

En análisis de datos, un uso típico de `for` es aplicar reglas similares a varias columnas.


In [ ]:
# Ejemplo 1: recorrer una lista de columnas numéricas para describirlas rápidamente
numeric_cols = ["age", "monthly_fee_clean", "orders_90d", "avg_order_value_usd", "total_spend_90d_usd",
                "support_tickets_90d", "satisfaction_score"]

for col in numeric_cols:
    # Ignoramos NaN automáticamente con pandas
    col_min = df[col].min()
    col_max = df[col].max()
    missing = df[col].isna().mean()
    print(f"{col:>20} | min={col_min:>8} | max={col_max:>8} | missing={missing:.3f}")


In [ ]:
# Ejemplo 2 (conceptual): un while para corregir valores fuera de rango (con un límite de iteraciones)
# Nota: normalmente esto se resuelve con reglas vectorizadas, pero lo usamos para entender la lógica del while.

ages = df["age"].copy()

# Definimos una regla: edades > 100 son inválidas, las convertimos a NaN.
# Repetimos hasta que no queden edades > 100 (o hasta un máximo de iteraciones por seguridad).
max_iters = 5
iters = 0

while (ages > 100).any() and iters < max_iters:
    ages = ages.mask(ages > 100, np.nan)  # convierte >100 a NaN
    iters += 1

print("Iteraciones realizadas:", iters)
print("¿Quedan edades > 100?:", (ages > 100).any())


### 7.1.5 Escribiendo tus primeras mini-funciones de limpieza de datos

Una mini-función de limpieza debe:

1. Recibir datos de entrada (por ejemplo un DataFrame).
2. Aplicar reglas claras (documentadas).
3. Devolver un resultado (DataFrame limpio o columnas limpias).
4. Ser fácil de reutilizar en otros datasets.

A continuación implementaremos dos funciones pequeñas:
- `clean_age(...)`: valida edad y decide cómo imputar.
- `clean_country_city(...)`: normaliza texto y maneja vacíos.


In [ ]:
def clean_age(df: pd.DataFrame, col: str = "age", min_age: int = 16, max_age: int = 80) -> pd.DataFrame:
    """Limpia una columna de edad.

    Reglas:
    - Convierte a numérico (si viniera como texto).
    - Edades fuera de [min_age, max_age] se vuelven NaN.
    - Imputa NaN con la mediana del dataset (simple y robusto ante outliers).

    Retorna:
    - Una copia del DataFrame con una nueva columna: f"{col}_clean"
    """
    out = df.copy()

    # 1) Convertir a numérico
    out[col] = pd.to_numeric(out[col], errors="coerce")

    # 2) Invalidar edades fuera de rango
    invalid_mask = (out[col] < min_age) | (out[col] > max_age)
    out.loc[invalid_mask, col] = np.nan

    # 3) Imputar con mediana (ignorando NaN)
    median_age = out[col].median()
    out[f"{col}_clean"] = out[col].fillna(median_age)

    return out

In [ ]:
def clean_country_city(df: pd.DataFrame, country_col: str = "country", city_col: str = "city") -> pd.DataFrame:
    """Normaliza columnas de país y ciudad y maneja vacíos.

    Reglas:
    - Quita espacios extra.
    - Estándar: primera letra mayúscula (title case).
    - Vacíos -> 'Unknown'
    """
    out = df.copy()

    for col in [country_col, city_col]:
        out[col] = out[col].fillna("").astype(str).str.strip()
        out[col] = out[col].replace("", "Unknown")
        out[col] = out[col].str.title()

    return out

In [ ]:
# Aplicar las mini-funciones
df_clean = clean_age(df)
df_clean = clean_country_city(df_clean)

df_clean[["age", "age_clean", "country", "city"]].head(10)

## Ejercicio 3 · Describiendo distribuciones de datos (7.2)

### 7.2.1 Medidas estadísticas en columnas numéricas

Para columnas numéricas solemos calcular:

- **Tendencia central**: media, mediana.
- **Dispersión**: desviación estándar, rango intercuartílico (IQR).
- **Percentiles**: p25, p50, p75, p90, p95.
- **Asimetría (skew)**: indica si hay cola larga a la derecha/izquierda (útil para decidir mediana vs media).

Empezaremos con un resumen numérico y luego lo interpretamos.


In [ ]:
# Resumen estadístico para numéricas (usando el DataFrame limpio)
num_cols = ["age_clean", "monthly_fee_clean", "orders_90d", "avg_order_value_usd",
            "total_spend_90d_usd", "support_tickets_90d", "satisfaction_score"]

summary_num = df_clean[num_cols].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95]).T
summary_num


### 7.2.2 Medidas estadísticas en columnas categóricas

En categóricas, lo más útil suele ser:

- **Frecuencias** y **proporciones** por categoría.
- **Moda** (categoría más frecuente).
- Cardinalidad (número de categorías distintas): demasiadas categorías suelen requerir normalización o agrupación.

Calculemos esto para `plan_type`, `segment`, `country`, `device_os` y `preferred_category`.


In [ ]:
cat_cols = ["plan_type", "segment", "country", "device_os", "preferred_category", "coupon_used"]

for col in cat_cols:
    counts = df_clean[col].value_counts(dropna=False)
    props = (counts / len(df_clean)).round(3)
    display(pd.DataFrame({"count": counts, "proportion": props}).head(10))


### 7.2.3 Visualizando distribuciones con histogramas

Un histograma permite ver:

- Concentración de valores (dónde se “acumulan”).
- Colas largas (posibles outliers o comportamientos raros).
- Si la distribución es aproximadamente normal o sesgada.

Graficaremos `total_spend_90d_usd` y `orders_90d`.


In [ ]:
# Histograma de gasto total (90 días)
plt.figure(figsize=(8,4))
plt.hist(df_clean["total_spend_90d_usd"], bins=40)
plt.title("Distribución: total_spend_90d_usd")
plt.xlabel("USD en 90 días")
plt.ylabel("Frecuencia")
plt.show()

In [ ]:
# Histograma de número de órdenes
plt.figure(figsize=(8,4))
plt.hist(df_clean["orders_90d"], bins=30)
plt.title("Distribución: orders_90d")
plt.xlabel("Órdenes en 90 días")
plt.ylabel("Frecuencia")
plt.show()

### 7.2.4 Visualizando distribuciones con boxplot

El boxplot resume una distribución con:

- Mediana (línea central).
- Cuartiles Q1 y Q3 (caja).
- IQR (Q3 - Q1).
- “Bigotes” y puntos extremos (posibles outliers según una regla).

Útil cuando quieres comparar distribuciones por grupo (por ejemplo: gasto por plan).


In [ ]:
# Boxplot: gasto por tipo de plan (comparación por grupo)
plt.figure(figsize=(9,4))

# Preparamos los datos por grupo (lista de series)
groups = []
labels = []
for p in df_clean["plan_type"].unique():
    groups.append(df_clean.loc[df_clean["plan_type"] == p, "total_spend_90d_usd"])
    labels.append(p)

plt.boxplot(groups, labels=labels, showfliers=True)
plt.title("Boxplot: total_spend_90d_usd por plan_type")
plt.xlabel("plan_type")
plt.ylabel("USD en 90 días")
plt.xticks(rotation=20)
plt.show()


### 7.2.5–7.2.6 Entendiendo el significado de negocio de distribuciones

Las distribuciones no son solo “formas”; cuentan una historia:

- Si `total_spend_90d_usd` es muy sesgada a la derecha: hay pocos clientes que aportan gran parte del ingreso (segmentación, VIP).
- Si `support_tickets_90d` tiene cola larga: hay un grupo pequeño de clientes con alta fricción (posible churn, issues de producto).
- Si `satisfaction_score` está concentrada en valores altos: el producto “en general” satisface; pero los extremos bajos son críticos.

Hagamos una lectura rápida por churn: comparar medias/medianas por `churned`.


In [ ]:
# Comparación simple por churn (0=activo, 1=churn)
metrics = ["total_spend_90d_usd", "orders_90d", "support_tickets_90d", "satisfaction_score", "monthly_fee_clean", "age_clean"]

group_stats = df_clean.groupby("churned")[metrics].agg(["mean", "median"])
group_stats


### Preguntas de reflexión sobre el Ejercicio 3

1. ¿Qué variables muestran diferencias más claras entre churned=0 y churned=1?  
2. Si el gasto es muy sesgado, ¿tiene sentido reportar “media” como valor típico? ¿Por qué?  
3. ¿Qué podría significar un aumento en la cola derecha de `support_tickets_90d`?  
4. ¿Qué decisión de negocio podrías tomar con base en los boxplots por plan?  


## Ejercicio 4 · Git/GitHub (primeros pasos) (8)

### ¿Por qué Git/GitHub para analistas?

- Git es un sistema de **control de versiones**: te permite registrar cambios, volver atrás y colaborar.
- GitHub es una plataforma para alojar repositorios Git y colaborar (issues, pull requests, etc.).
- En proyectos de analytics, Git ayuda a:
  - Versionar notebooks/scripts y documentación.
  - Compartir análisis de manera reproducible.
  - Mantener historial de cambios (auditabilidad).

### 8.1 Instalar Git (Windows / macOS)

**Windows**
1. Descarga “Git for Windows” e instala con opciones por defecto.
2. Verifica en una terminal (PowerShell o CMD):  
   `git --version`

**macOS**
1. Abre Terminal y ejecuta:  
   `git --version`  
   Si no está instalado, macOS suele proponer instalar “Command Line Tools”.
2. Verifica: `git --version`

### 8.2 Crear cuenta en GitHub (checklist)
- Crear usuario.
- Activar verificación en dos pasos (recomendado).
- Configurar una llave SSH (opcional al inicio) o usar HTTPS.

### 8.3 Flujo mínimo recomendado (para el curso)
1. Crear un repositorio en GitHub.
2. Clonarlo localmente: `git clone <url>`
3. Guardar tu notebook y dataset dentro del repo.
4. `git add .`
5. `git commit -m "Primer commit: Webinar 19"`
6. `git push`

> En sesiones prácticas, aplicaremos este flujo con notebooks y datasets pequeños para que se vuelva hábito.


## 6. Take aways de la sesión teórica

- “Preparar datos” significa: validar tipos, tratar faltantes e inválidos y documentar decisiones.
- Antes de limpiar, define el objetivo: tus reglas dependen del contexto de negocio.
- Usa funciones de Pandas (`to_numeric`, `to_datetime`, `fillna`, `value_counts`) como primera línea de trabajo.
- Los bucles ayudan a automatizar tareas repetitivas, pero en Pandas casi siempre preferimos operaciones vectorizadas.
- Escribir mini-funciones te obliga a ser claro/a con reglas y favorece la reutilización.
- Las distribuciones te ayudan a entender comportamiento de clientes y tomar decisiones: segmentación, priorización de soporte, prevención de churn.
- Git/GitHub convierte tu trabajo en un “proyecto” con historial y colaboración.


## 7. Cierre y próximos pasos

- Revisa el dataset y busca 3 problemas adicionales de calidad de datos que no hayamos discutido.
- Propón (en texto) una regla de limpieza por cada problema identificado.
- Como preparación para la próxima sesión:
  - Practica `describe()`, `value_counts()` y al menos 2 gráficos con `matplotlib`.
  - Si puedes, instala Git y crea tu cuenta de GitHub para usarla en el siguiente webinar.


## 8. Información complementaria y recursos

- Documentación de Pandas: tipos de datos, missing values, `to_datetime`, `to_numeric`.
- Buenas prácticas de análisis exploratorio (EDA): selección de variables, validación de reglas.
- Git/GitHub: conceptos de repositorio, commit, push, branch (lo veremos paso a paso en práctica).

Sugerencia: conserva este notebook como plantilla para futuros proyectos; cambia el dataset y reusa la estructura.
